<a href="https://colab.research.google.com/github/lucasdsbarreto/IA-analise-BK-sistemasNeSy/blob/main/Trabalho_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo CILP
Experimentos com exemplos positivos e negativos


## Bibliotecas

In [1]:
import math
import random
import numpy as np

## Class Neural Network

In [2]:
def predict (valores):
  predicoes = [1 if v >=0.5 else 0 for v in valores]
  return predicoes

In [3]:

random.seed(0)
def rand(a, b):
  return (b-a)*random.random() + a

def sigmoid(x, diff=False):
  if diff:
    return 1.0 - x**2
  else:
    return math.tanh(x)

class NN:
  def __init__(self, ni, nh, no, pesos, w):
    self.ni = ni + 1 
    self.nh = nh 
    self.no = no

    self.ai = [1.0]*self.ni
    self.ah = [1.0]*self.nh
    self.ao = [1.0]*self.no


    self.wi = np.zeros((self.ni, self.nh))
    self.wo = np.zeros((self.nh, self.no))

    self.wi = pesos

    for j in range(self.nh):
      for k in range(self.no):
        self.wo[j][k] = rand(-0.2, 0.2)

    self.ci = np.zeros((self.ni, self.nh))
    self.co = np.zeros((self.nh, self.no))

  def update(self, inputs):
    if len(inputs) != self.ni-1:
      raise ValueError('input error')


    for i in range(self.ni-1):
      self.ai[i] = inputs[i]

    for j in range(self.nh):
      sum = 0.0
      for i in range(self.ni):
          sum = sum + self.ai[i] * self.wi[i][j]
      self.ah[j] = sigmoid(sum)

    for k in range(self.no):
      sum = 0.0
      for j in range(self.nh):
          sum = sum + self.ah[j] * self.wo[j][k]
      self.ao[k] = sigmoid(sum)

    return self.ao[:]


  def backpropagation(self, targets, N, M):
    if len(targets) != self.no:
      raise ValueError('wrong number of target values')

    output_deltas = [0.0] * self.no
    for k in range(self.no):
      error = targets[k]-self.ao[k]
      output_deltas[k] = sigmoid(self.ao[k], diff=True) * error

    hidden_deltas = [0.0] * self.nh
    for j in range(self.nh):
      error = 0.0
      for k in range(self.no):
          error = error + output_deltas[k]*self.wo[j][k]
      hidden_deltas[j] = sigmoid(self.ah[j], diff=True) * error

    for j in range(self.nh):
      for k in range(self.no):
        change = output_deltas[k]*self.ah[j]
        self.wo[j][k] = self.wo[j][k] + N*change + M*self.co[j][k]
        self.co[j][k] = change

    for i in range(self.ni):
      for j in range(self.nh):
        change = hidden_deltas[j]*self.ai[i]
        self.wi[i][j] = self.wi[i][j] + N*change + M*self.ci[i][j]
        self.ci[i][j] = change

    error = 0.0
    for i in range(len(targets)):
      error = error + 0.5*(targets[i]-self.ao[i])**2

    return error


  def test(self, patterns):
    for p in patterns: print(p[0], '- predição ->', predict(self.update(p[0])),"resultado esperado ->",p[1])

  def weights(self):
    print(' Input weights:')
    for i in range(self.ni): print(self.wi[i])

    print('\n Output weights:')
    for j in range(self.nh): print(self.wo[j])

  def train(self, patterns, iterations=2598, N=0.5, M=0.1):
    for i in range(iterations):

      error = 0.0
      for p in patterns:
        inputs = p[0]
        targets = p[1]
        self.update(inputs)
        error = error + self.backpropagation(targets, N, M)


## CILP

## Funções principais

In [4]:

def funcH(x,b):
  aux = math.exp(-b*x)
  return ((2/(1 + aux)) - 1)


# use to know a negation variable
def isNot(x, ver=False):
  
  if ver:
    if x!='':
      if x[0]=="-":
        return True
      else: return False
    else: return False

  else:
    if x[0]=="-":
      return x[1]
    else:
      return "-"+x

# get litearais
def literais(P):
  all = []
  for r in P:
    for s in r:
      for t in s:
        if (t not in all) and (t != '') and (isNot(t) not in all):
          if isNot(t,True):
            all.append(isNot(t))
          else:
            all.append(t)
  return all


def neg_pos(P):
  Neg = []
  Pos = []
  for k in P:
    n=0
    p=0
    for r in k[1]:
      if isNot(r,True):
        print("neg: "+r)
        n+=1
      else:
        if r!='':
          print("pos:  "+r)
          p+=1

    Neg.append(n)
    Pos.append(p)
    print()
  return Neg,Pos

# get values from data
def q_v_k(P):
  q = len(P)
  v = len(literais(P))
  K = []
  k = 0
  for r in P:
    if r[1]!=['']:
      K.append(len(r[1]))
    else:
      K.append(0)

  return q,v,K

def u_heads(P):
  heads = [a[0] for a in P]
  return [heads.count(a) for a in heads]

# variable MAXp
def Maxr(k,u):
  if k>u: return k
  else: return u

def Maxp(k,u):
  max = 0
  for i in k:
    for j in u:
      if Maxr(i,j)>max:
        max = Maxr(i,j)
  return max

# variable Amin
def Amin(k,u):
  return ((Maxp(k,u)-1) / (Maxp(k,u)+1))

def W_pesos(k,u,b, amin):
  return (2/b)*(math.log(1+amin)-math.log(1-amin))/(Maxp(k,u)*(amin-1) + amin+1)

# hidden layer
def threshold_hidden(amin, K, W):
  return [(1+amin)*(k-1)*W/2 for k in K]

def empyt(x):
  if x=='':
    return 0
  else: return 1

In [5]:
def W_pesos(k,u,b, amin):
  return (2/b)*(math.log(1+amin)-math.log(1-amin))/(Maxp(k,u)*(amin-1) + amin+1)


In [6]:
def negacoes(c):
  n = c[1]
  for i in range(len(n)):
    if isNot(n[i], True):
      n[i]=isNot(n[i])
  return n

In [7]:

def pesos_clauses(P, w, amin):
  pesos = []
  for r in P:
    l = literais(P)
    l.append('Bias')
    dicio = dict.fromkeys(l, 0)
    try:
        while True:
            r[1].remove('')
    except ValueError:
        pass
    values = [-1 if isNot(a, True) else empyt(a) for a in r[1]]
    keys = negacoes(r)
    for i in range(len(keys)):
      dicio[keys[i]] = values[i]
    pesos.append(list(dicio.values()))
  
  for i in range(len(pesos)):
    for a in range(len(pesos[i])):
      # print(a)
      pesos[i][a]=pesos[i][a]*w
  pesos = np.transpose(pesos)
  q, v, K = q_v_k(P)
  pesos[-1] = threshold_hidden(amin, K, w)
  return pesos

##Experimento 1

-> Clausulas Escolhidas para Experimento

hd(X) :- female(X), parent(X,Y), female(Y).

hd(X) :- male(X), parent(Y,X).

hd(X) :- female(X), parent(Y,X), male(Y).

hd(X) :- male(X), parent(X,Y), female(Y).

hd(X) :- female(X), parent(Y,X).

hd(X) :- male(X), parent(X,Y).



->Relação entre as regras para literais

A : hd(X)

B : female(X)

C : male(X)

D : parent(X,Y)

E : female(Y)

F : male(Y)

G : parent(Y,X)

H : male(Y)


-> Cláusulas como literais

A :- B, D, E.

A :- C, G.

A :- B, G, H.

A :- C, D, E.

A :- B, G.

A :- C, D.

###Treino

In [8]:

# Coloca a base de treino aqui
input = [      
        #tom
        [[0,1,1,1,1,0,0], [1]],
        #pam
        # [[1,0,1,0,1,0,0], [0]],
        #liz
        [[1,0,0,0,0,1,1], [0]],
        #bob
        # [[0,1,1,1,0,1,1], [1]],
        #ann
        [[1,0,1,1,0,1,1], [1]],
        #pat
        [[1,0,1,0,1,1,1], [0]],
        #jim
        [[0,1,0,0,0,1,0], [0]],
        #eve
        [[1,0,0,0,0,1,0], [0]],
        
]


P = [['A', ['B', 'D','E']], ['A', ['-C', '-G']], ['A', ['-B', '-G','-H']], ['A', ['C', 'D','E']], ['A', ['-B', '-G']], ['A', ['-E', '-G']]]

q, v, K = q_v_k(P)
U = u_heads(P)
amin = (1 - Amin(K,U))/2 + Amin(K,U)
w = W_pesos(K, U, 1, amin)

pesos = pesos_clauses(P,w,amin)
print("Numero de Entradas:",v)
print("Numero de Camadas Ocultas:",q)
# Estrutura da rede
  # n entradas
  # n camadas invisíveis
  # n saídas 
  # lista de pesos
  # W calculado
n = NN(v, q, 1, pesos, w)

#Treino
n.train(input)


Numero de Entradas: 7
Numero de Camadas Ocultas: 6


###Teste

In [9]:

#Teste
output = [      
        #pam
        [[1,0,1,0,1,0,0],[0]],
        #bob
        [[0,1,1,1,0,1,1],[1]],
        #pat
        [[1,0,1,0,1,1,1],[0]],
]
n.test(output)

[1, 0, 1, 0, 1, 0, 0] - predição -> [0] resultado esperado -> [0]
[0, 1, 1, 1, 0, 1, 1] - predição -> [1] resultado esperado -> [1]
[1, 0, 1, 0, 1, 1, 1] - predição -> [0] resultado esperado -> [0]


##Experimento 2

-> Clausulas Escolhidas para Experimento

hd(X) :- female(X), parent(X,Y), female(Y).

hd(X) :- male(X), parent(X,Y), female(Y).


->Relação entre as regras para literais

A : hd(X)

B : female(X)

C : male(X)

D : parent(X,Y)

E : female(Y)

F : male(Y)


-> Cláusulas como literais

A :- B, D, E.

A :- C, D, E.

###Treino

In [10]:

# Coloca a base de treino aqui
input = [      
        #tom
        [[0,1,1,1,1], [1]],
        #pam
        # [[1,0,1,0,1], [0]],
        #liz
        [[1,0,0,0,0], [0]],
        #bob
        [[0,1,1,1,0], [1]],
        #ann
        [[1,0,1,1,0], [1]],
        #pat
        [[1,0,1,0,1], [0]],
        #jim
        [[0,1,0,0,0], [0]],
        #eve
        [[1,0,0,0,0], [0]],
        
]


P = [['A', ['B', 'D','E']], ['A', ['C', 'D','E']]]

q, v, K = q_v_k(P)
U = u_heads(P)
amin = (1 - Amin(K,U))/2 + Amin(K,U)
w = W_pesos(K, U, 1, amin)
pesos = pesos_clauses(P,w,amin)

print("Numero de Entradas:",v)
print("Numero de Camadas Ocultas:",q)

# Estrutura da rede
  # n entradas
  # n camadas invisíveis
  # n saídas 
  # lista de pesos
  # W calculado
n = NN(v, q, 1, pesos, w)

#Treino
n.train(input)


Numero de Entradas: 5
Numero de Camadas Ocultas: 2


###Teste

In [11]:

#Teste
output = [      
        #pam
        [[1,0,1,0,1],[0]],
        #bob
        [[0,1,1,1,0],[1]],
        #pat
        [[1,0,1,0,1],[0]],
]
n.test(output)

[1, 0, 1, 0, 1] - predição -> [0] resultado esperado -> [0]
[0, 1, 1, 1, 0] - predição -> [0] resultado esperado -> [1]
[1, 0, 1, 0, 1] - predição -> [0] resultado esperado -> [0]
